In [ ]:
import numpy as np, os, random, re, pickle
from PIL import Image
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
# sort files as 1,2,3,4,... and not as 1,10,100,11,...
def sort_nicely(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort( key=alphanum_key )

In [ ]:
dir_1 = # where the SMPL-X parameters of kostas' images are saved
dir_2 = # where the SMPL-X parameters of olga's  images are saved

classes = np.load('classes.npy')

In [ ]:
x, y_n = [], []
for c in classes:
    for dir in os.listdir(dir_1):
        if c==dir:
            x.extend([dir_1+dir+'/'+f for f in os.listdir(dir_1+dir)])
            length = len([f for f in os.listdir(dir_1+dir)])
            y_n.extend([dir]*length)
            
for c in classes:
    for dir in os.listdir(dir_2):
        if c==dir:
            x.extend([dir_2+dir+'/'+f for f in os.listdir(dir_2+dir)])
            length = len([f for f in os.listdir(dir_2+dir)])
            y_n.extend([dir]*length)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y_n)
x = np.array(x)

In [ ]:
X_train, X_dev, X_test = [], [], []
Y_train, Y_dev, Y_test = [], [], []
for i in range(0,max(y)+1):
    indices = [j for j in range(len(y)) if y[j]==i]; random.shuffle(indices)
    train = indices[0:int(len(indices)*0.6)]
    dev   = indices[int(0.6*len(indices)):int(0.8*len(indices))]
    test  = indices[int(0.8*len(indices)):len(indices)]
    X_train.extend(x[train]); Y_train.extend([i]*len(train))
    X_dev.extend(x[dev]); Y_dev.extend([i]*len(dev))
    X_test.extend(x[test]); Y_test.extend([i]*len(test))

In [ ]:
train_feat = []
for x in X_train:
    print(x)
    break
    images = [x+'/results/'+f for f in os.listdir(x+'/results/')]; sort_nicely(images)
    for im in images:
        with open(im+'/'+os.listdir(im)[0], 'rb') as f: data = pickle.load(f)
        features.append(np.squeeze(np.concatenate((data['betas'],data['global_orient'],
                                                   data['left_hand_pose'],data['right_hand_pose'],
                                                   data['jaw_pose'], data['leye_pose'],
                                                   data['reye_pose'],data['expression'],data['body_pose']),1)))
    train_feat.append(np.array(features))
for x in X_dev:
    features = []
    images = [x+'/results/'+f for f in os.listdir(x+'/results/')]; sort_nicely(images)
    for im in images:
        with open(im+'/'+os.listdir(im)[0], 'rb') as f: data = pickle.load(f)
        features.append(np.squeeze(np.concatenate((data['betas'],data['global_orient'],
                                                   data['left_hand_pose'],data['right_hand_pose'],
                                                   data['jaw_pose'], data['leye_pose'],
                                                   data['reye_pose'],data['expression'],data['body_pose']),1)))
    dev_feat.append(np.array(features))

test_feat = []
for x in X_test:
    features = []
    images = [x+'/results/'+f for f in os.listdir(x+'/results/')]; sort_nicely(images)
    for im in images:
        with open(im+'/'+os.listdir(im)[0], 'rb') as f: data = pickle.load(f)
        features.append(np.squeeze(np.concatenate((data['betas'],data['global_orient'],
                                                   data['left_hand_pose'],data['right_hand_pose'],
                                                   data['jaw_pose'], data['leye_pose'],
                                                   data['reye_pose'],data['expression'],data['body_pose']),1)))
    test_feat.append(np.array(features))

In [ ]:
scaler = StandardScaler()

for i in range(len(train_feat)):
    train_feat[i] = scaler.fit_transform(train_feat[i])
    
for i in range(len(dev_feat)):
    dev_feat[i] = scaler.fit_transform(dev_feat[i])
    
for i in range(len(test_feat)):
    test_feat[i] = scaler.fit_transform(test_feat[i])

In [ ]:
np.save('data/x_dev_sm.npy',dev_feat); np.save('data/y_dev_sm.npy',Y_dev)
np.save('data/x_test_sm.npy',test_feat); np.save('data/y_test_sm.npy',Y_test)
np.save('data/x_train_sm.npy',train_feat); np.save('data/y_train_sm.npy',Y_train)